<a href="https://colab.research.google.com/github/Abhishek-IITH/Projects/blob/main/Beacon_tracking/Navigation_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import serial
import RPi.GPIO as GPIO
import pandas as pd
import datetime

ser = serial.Serial(port = '/dev/ttyUSB0',
                    baudrate = 115200,
                    parity = serial.PARITY_NONE,
                    stopbits = serial.STOPBITS_ONE,
                    bytesize=serial.EIGHTBITS,
                    timeout=1)
servoPIN = 12
IN1 = 16
IN2 = 20
IN3 = 21
IN4 = 26
speed_control1 = 13
speed_control2 = 18
GPIO.setwarnings(False)
GPIO.setmode(GPIO.BCM)

GPIO.setup(servoPIN, GPIO.OUT)
GPIO.setup(IN1, GPIO.OUT)
GPIO.setup(IN2, GPIO.OUT)
GPIO.setup(IN3, GPIO.OUT)
GPIO.setup(IN4, GPIO.OUT)
GPIO.setup(speed_control1, GPIO.OUT)
GPIO.setup(speed_control2, GPIO.OUT)
s1 = GPIO.PWM(speed_control1, 490)
s2 = GPIO.PWM(speed_control2, 490)
s1.start(37)
s2.start(35)
rssi_arr = [['t',0,0,0,0,0]]*15
df = pd.DataFrame(columns=['Time','0','45','90','135','180'])

#Function to read single RSSI value from ESP32
def check_RSSI():
  rssi = 0 
  for i in range(15):
    while ser.in_waiting == 0:
        continue
    rssi = rssi+ int(ser.readline().decode('ascii').rstrip())
    print(rssi)
  return rssi/15

#Function to Rotate the ESP32 in five angles using servo and reading RSSI values in those angles
def Get_RSSI():
    p = GPIO.PWM(servoPIN, 50) # GPIO 17 for PWM with 50Hz
    #rssi_arr[0]=datetime.datetime.now()
    p.start(2.5)
    rssi_arr[0]= check_RSSI()
    time.sleep(0.5)# Initialization
    # for i in range(15):
    #     rssi_arr[i][1]=check_RSSI()
        #rssi_arr[0]=datetime.datetime.now()
    # time.sleep(0.5)
    for i in range(1,5):
        p.ChangeDutyCycle(2.5*i)
        rssi_arr[i]=check_RSSI()
        time.sleep(0.5)
    # for j in range(15):
        #     rssi_arr[j]=check_RSSI()
        #time.sleep(0.5)
    p.ChangeDutyCycle(7.5)
    print(rssi_arr)
    time.sleep(1)

def dir_finder():
    arr3_m = (rssi_arr[0]+rssi_arr[1]+rssi_arr[2])/3 
    arr3_n = (rssi_arr[1]+rssi_arr[2]+rssi_arr[3])/3
    arr3_o = (rssi_arr[2]+rssi_arr[3]+rssi_arr[5])/3
    arr = [arr3_m ,arr3_n , arr3_o]
    dir = arr.index(max(arr))
    return dir

def angle_finder(dir):
  if dir==0:
    left_turn()
    stop(0.5)
    forward(30)
    stop(0.5)
  if dir ==2:
    right_turn()
    stop(0.5)
    forward(30)
    stop(0.5)
  if dir ==1:
    forward(30)
    stop(0.5)

        
    
# Function to add the new datapoint into the CSV file
def add2csv(rssi_arr):
    df = pd.read_csv('test.csv')
    for i in range(15):
        df.loc[df.shape[0]] = rssi_arr[i]
    df.to_csv('test.csv', index = False)

def forward(distcm):
    GPIO.output(IN1,0)
    GPIO.output(IN2,1)
    GPIO.output(IN3,0)
    GPIO.output(IN4,1)
    time.sleep(distcm/29)
    
def stop(t):
    GPIO.output(IN1,0)
    GPIO.output(IN2,0)
    GPIO.output(IN3,0)
    GPIO.output(IN4,0)
    time.sleep(t)
#for i in range(3):
    #Get_RSSI()
   # print(rssi_arr)
   # add2csv(rssi_arr)
def right_turn():
    GPIO.output(IN1,0)
    GPIO.output(IN2,0)
    GPIO.output(IN3,1)
    GPIO.output(IN4,0)
    time.sleep(0.92)

def left_turn():
    GPIO.output(IN1,1)
    GPIO.output(IN2,0)
    GPIO.output(IN3,0)
    GPIO.output(IN4,0)
    time.sleep(1.05)
#time.sleep(3)
#function to rotate in a grid-------------------
    
def grid_rotate():
    for i in range(4):
        #right_turn()
        stop(1)
        Get_RSSI()
        add2csv(rssi_arr)
        stop(1)
# main-----------------------------------
#for i in range():
    #forward(60)
    #stop(1)
#grid_rotate()
for i in range(1):
  Get_RSSI()
  dir_finder()
  angle_finder()
  add2csv(rssi_arr)